# Short Selling Example

This notebook demonstrates how to use short selling strategies in the trading system.

## Overview

Short selling allows you to profit from declining prices by:
- Selling borrowed shares at current price
- Buying them back later at (hopefully) lower price
- Profiting from the price difference

The trading system supports short selling through:
- **Mean Reversion Strategy**: Shorts when assets are overbought (z-score > entry_std)
- **Pairs Trading Strategy**: Long-short pairs for market-neutral positions
- **Portfolio Management**: Separate limits for long/short exposure

## Key Concepts

- **Short Entry**: Sell when price is expected to decline
- **Short Exit**: Buy back when price rebounds or stop loss triggered
- **Stop Loss**: For shorts, stop is ABOVE entry price (losses if price rises)
- **Borrow Costs**: Interest charged on borrowed shares (configurable)


In [ ]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Add project root to path
project_root = Path().resolve().parent.parent.parent
sys.path.insert(0, str(project_root))

from trading_system.integration.runner import run_backtest
from trading_system.configs.run_config import RunConfig
from trading_system.reporting.metrics import MetricsCalculator

print("Setup complete!")


## Example 1: Mean Reversion Strategy (Short Selling)

The mean reversion strategy generates short signals when assets are overbought (z-score > entry_std).

**Entry Logic:**
- Calculate z-score: (price - rolling_mean) / rolling_std
- Short when z-score > entry_std (e.g., z-score > 2.0 means price is 2 standard deviations above mean)
- Exit when z-score returns to mean (z-score <= exit_std, typically 0.0)

**Stop Loss:**
- For shorts, stop is ABOVE entry price
- Stop = entry_price + (stop_atr_mult * ATR14)
- If price rises above stop, position is closed at a loss


In [ ]:
# Load run configuration with mean reversion strategy
config_path = project_root / "EXAMPLE_CONFIGS" / "mean_reversion_config.yaml"

if not config_path.exists():
    print(f"Config not found: {config_path}")
    print("Using test config instead...")
    config_path = project_root / "tests" / "fixtures" / "configs" / "run_test_config.yaml"

# Load run config
run_config_path = project_root / "EXAMPLE_CONFIGS" / "run_config.yaml"
run_config = RunConfig.from_yaml(str(run_config_path))

# Modify run config to use mean reversion strategy
from trading_system.configs.run_config import StrategyConfig

# Update strategy config to use mean reversion
run_config.strategies.equity.config_path = str(config_path)
run_config.strategies.equity.enabled = True

print(f"Using mean reversion config: {config_path}")
print(f"Universe: {run_config.strategies.equity.universe if hasattr(run_config.strategies.equity, 'universe') else 'From config'}")


In [ ]:
# Run backtest with mean reversion strategy
print("Running backtest with mean reversion (short selling) strategy...")
print("This may take a minute...")

try:
    results = run_backtest(
        config_path=None,  # Use config object directly
        period="train",
        config_obj=run_config
    )
    
    print(f"\nBacktest completed!")
    print(f"Results saved to: {results.output_dir}")
    
except Exception as e:
    print(f"Error running backtest: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Analyze results - check for short positions
results_dir = results.output_dir if 'results' in locals() else None

if results_dir and Path(results_dir).exists():
    trade_log_path = Path(results_dir) / "trade_log.csv"
    equity_curve_path = Path(results_dir) / "equity_curve.csv"
    
    if trade_log_path.exists():
        trades_df = pd.read_csv(trade_log_path)
        
        # Check for short positions (side == 'SHORT')
        if 'side' in trades_df.columns:
            short_trades = trades_df[trades_df['side'] == 'SHORT']
            long_trades = trades_df[trades_df['side'] == 'LONG']
            
            print(f"Total trades: {len(trades_df)}")
            print(f"Long trades: {len(long_trades)}")
            print(f"Short trades: {len(short_trades)}")
            
            if len(short_trades) > 0:
                print(f"\nShort Trade Summary:")
                print(f"  Average return: {short_trades['return'].mean():.2%}")
                print(f"  Win rate: {(short_trades['return'] > 0).mean():.2%}")
                print(f"  Average hold days: {short_trades['hold_days'].mean():.1f}")
        else:
            print("Trade log doesn't have 'side' column - positions may all be long")
    else:
        print(f"Trade log not found: {trade_log_path}")
else:
    print("Results directory not available")


## Understanding Short Position P&L

For short positions, P&L works in reverse:

- **Entry**: Sell shares at entry_price → Cash INCREASES
- **Exit**: Buy back shares at exit_price → Cash DECREASES
- **Profit**: If exit_price < entry_price, you profit (bought back cheaper)
- **Loss**: If exit_price > entry_price, you lose (bought back more expensive)

**Stop Loss for Shorts:**
- Stop is placed ABOVE entry price
- If price rises to stop, position is closed at a loss
- Example: Entry at $100, stop at $102 → 2% loss if triggered


In [ ]:
# Visualize short vs long performance
if results_dir and Path(results_dir).exists():
    trade_log_path = Path(results_dir) / "trade_log.csv"
    
    if trade_log_path.exists():
        trades_df = pd.read_csv(trade_log_path)
        
        if 'side' in trades_df.columns and len(trades_df[trades_df['side'] == 'SHORT']) > 0:
            fig, axes = plt.subplots(2, 1, figsize=(12, 10))
            
            # Plot 1: Returns distribution
            short_returns = trades_df[trades_df['side'] == 'SHORT']['return']
            long_returns = trades_df[trades_df['side'] == 'LONG']['return']
            
            axes[0].hist(long_returns, bins=30, alpha=0.7, label='Long', color='green')
            axes[0].hist(short_returns, bins=30, alpha=0.7, label='Short', color='red')
            axes[0].axvline(0, color='black', linestyle='--', linewidth=1)
            axes[0].set_xlabel('Return (%)')
            axes[0].set_ylabel('Frequency')
            axes[0].set_title('Returns Distribution: Long vs Short')
            axes[0].legend()
            axes[0].grid(True, alpha=0.3)
            
            # Plot 2: Cumulative returns by side
            trades_df['cumulative_return'] = trades_df.groupby('side')['return'].cumsum()
            
            for side in ['LONG', 'SHORT']:
                side_trades = trades_df[trades_df['side'] == side].copy()
                if len(side_trades) > 0:
                    side_trades = side_trades.sort_values('exit_date')
                    axes[1].plot(side_trades['exit_date'], side_trades['cumulative_return'], 
                               label=f'{side} Cumulative Return', marker='o', markersize=3)
            
            axes[1].axhline(0, color='black', linestyle='--', linewidth=1)
            axes[1].set_xlabel('Date')
            axes[1].set_ylabel('Cumulative Return (%)')
            axes[1].set_title('Cumulative Returns Over Time')
            axes[1].legend()
            axes[1].grid(True, alpha=0.3)
            axes[1].tick_params(axis='x', rotation=45)
            
            plt.tight_layout()
            plt.show()
        else:
            print("No short positions found in results")
    else:
        print("Trade log not found")
else:
    print("Results not available")


## Risk Management for Short Positions

The system includes several risk controls for short positions:

1. **Separate Exposure Limits:**
   - `max_long_exposure`: Maximum long exposure (% of equity)
   - `max_short_exposure`: Maximum short exposure (% of equity)
   - `max_net_exposure`: Maximum net exposure (long - short)

2. **Borrow Costs:**
   - Interest charged on borrowed shares
   - Configurable in execution costs
   - Reduces returns on short positions

3. **Stop Losses:**
   - Critical for shorts (unlimited loss potential without stops)
   - Stops placed above entry price
   - Tight stops recommended for volatile assets

4. **Correlation Guards:**
   - Prevents over-concentration in correlated shorts
   - Works for both long and short positions


In [ ]:
# Check portfolio exposure and risk metrics
if results_dir and Path(results_dir).exists():
    equity_curve_path = Path(results_dir) / "equity_curve.csv"
    
    if equity_curve_path.exists():
        equity_df = pd.read_csv(equity_curve_path, parse_dates=['date'])
        
        # Check for exposure columns (may vary by implementation)
        exposure_cols = [col for col in equity_df.columns if 'exposure' in col.lower()]
        
        if exposure_cols:
            print("Exposure Analysis:")
            for col in exposure_cols:
                print(f"  {col}: max={equity_df[col].max():.2f}, mean={equity_df[col].mean():.2f}")
        
        # Plot equity curve
        plt.figure(figsize=(12, 6))
        plt.plot(equity_df['date'], equity_df['equity'], label='Portfolio Equity', linewidth=2)
        plt.xlabel('Date')
        plt.ylabel('Equity ($)')
        plt.title('Portfolio Equity Curve (Mean Reversion Strategy)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
        # Summary statistics
        total_return = (equity_df['equity'].iloc[-1] / equity_df['equity'].iloc[0] - 1) * 100
        max_equity = equity_df['equity'].max()
        min_equity = equity_df['equity'].min()
        max_drawdown = ((max_equity - equity_df['equity'].iloc[equity_df['equity'].idxmin()]) / max_equity) * 100
        
        print(f"\nPerformance Summary:")
        print(f"  Total Return: {total_return:.2f}%")
        print(f"  Max Drawdown: {max_drawdown:.2f}%")
        print(f"  Final Equity: ${equity_df['equity'].iloc[-1]:,.2f}")
    else:
        print("Equity curve not found")
else:
    print("Results not available")


## Summary

This notebook demonstrated:

1. **Mean Reversion Short Selling**: Using z-scores to identify overbought conditions
2. **Short Position Mechanics**: How P&L works for short positions
3. **Risk Management**: Exposure limits, borrow costs, and stop losses
4. **Performance Analysis**: Comparing long vs short trade performance

## Next Steps

- Explore **Pairs Trading** strategy for market-neutral long-short pairs
- Experiment with different entry/exit thresholds in mean reversion config
- Review risk limits in configuration files
- Consider borrow costs when evaluating short strategy profitability

## Additional Resources

- Mean Reversion Config: `EXAMPLE_CONFIGS/mean_reversion_config.yaml`
- Pairs Trading: See pairs trading example notebook (coming soon)
- Risk Configuration: `agent-files/05_PORTFOLIO_AND_RISK.md`
